In [ ]:
pip install langchain python-dotenv langchain-groq tiktoken rapidocr-onnxruntime langchain-community langchain-text-splitters langchain-huggingface sentence-transformers

In [9]:
from langchain_groq import ChatGroq

model=ChatGroq(model="llama-3.1-8b-instant", temperature=0)
response=model.invoke("HI how are you")
print(response.content)

I'm just a computer program, so I don't have feelings, but I'm functioning properly and ready to help you with any questions or tasks you have. How can I assist you today?


In [ ]:
pip install -qU langchain-chroma

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
pip install pypdf

In [52]:
from langchain_community.document_loaders import PyPDFLoader
import pprint

loader=PyPDFLoader("/Users/adityaaryan/Documents/mlops/data/Saathi_Schemes_Info.pdf")
docs=loader.load()
for doc in docs:
    pprint.pprint({
        "page": doc.metadata.get("page"),
        "content": doc.page_content[:500]  # first 500 chars
    })


{'content': 'Saathi: Government Scheme Information Handbook  \n'
            '1. Pradhan Mantri Awas Yojana (PMAY) \n'
            'About the Scheme: \n'
            'An initiative by the Government of India aimed at providing '
            'affordable housing for all by the \n'
            'year 2025. It targets urban and rural poor populations with '
            'financial assistance to build or \n'
            'purchase a home. \n'
            'Eligibility Criteria: \n'
            'Indian citizen aged 18 or above \n'
            'Annual household income below ₹18 lakhs \n'
            'Beneficiary should not own a pucca house in any part of India \n'
            'Priority given to women, SC',
 'page': 0}
{'content': 'Why is it beneficial? \n'
            'Provides financial protection from catastrophic health expenses '
            'and reduces out-of-pocket \n'
            'healthcare costs for the poor. \n'
            ' \n'
            '3. Pradhan Mantri Kisan Samman Nidhi (PM-K

In [77]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
split_docs=text_splitter.split_documents(docs)
split_docs

[Document(metadata={'page': 0, 'source': '/Users/adityaaryan/Documents/mlops/data/Saathi_Schemes_Info.pdf', 'moddate': '2025-06-22T12:19:11+00:00', 'creator': 'Microsoft® Word 2016', 'page_label': '1', 'author': 'python-docx', 'producer': 'www.ilovepdf.com', 'total_pages': 3, 'creationdate': '2025-06-22T12:19:11+00:00'}, page_content='Saathi: Government Scheme Information Handbook  \n1. Pradhan Mantri Awas Yojana (PMAY) \nAbout the Scheme: \nAn initiative by the Government of India aimed at providing affordable housing for all by the'),
 Document(metadata={'total_pages': 3, 'source': '/Users/adityaaryan/Documents/mlops/data/Saathi_Schemes_Info.pdf', 'page_label': '1', 'creator': 'Microsoft® Word 2016', 'producer': 'www.ilovepdf.com', 'creationdate': '2025-06-22T12:19:11+00:00', 'page': 0, 'moddate': '2025-06-22T12:19:11+00:00', 'author': 'python-docx'}, page_content='Saathi: Government Scheme Information Handbook  \n1. Pradhan Mantri Awas Yojana (PMAY) \nAbout the Scheme: \nAn initiati

In [78]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,   # Embedding function to convert text to vectors
    persist_directory="../langchain_db",  # Where to save data locally
)

In [79]:
vector_store.add_documents(split_docs)


['f44c6d2a-9fa6-4c54-8d9f-08b7acee3740',
 '5e694f1c-9608-4c37-ab5b-401e0231e319',
 'bfdbc76a-a0f3-43e1-8510-7ff17f97e47a']

In [93]:
# Method 1 - Using retriever (better method)
retriever=vector_store.as_retriever(search_kwargs={"k": 5}) # Number of similar documents to retrieve
question = "What is PMAY scheme?"
docs = retriever.invoke(question)
docs

[Document(id='96282d8a-e3be-417c-8de2-1d75e81a81f7', metadata={'moddate': '2025-06-22T12:19:11+00:00', 'source': '/Users/adityaaryan/Documents/mlops/data/Saathi_Schemes_Info.pdf', 'creationdate': '2025-06-22T12:19:11+00:00', 'page': 0, 'creator': 'Microsoft® Word 2016', 'page_label': '1', 'producer': 'www.ilovepdf.com', 'author': 'python-docx', 'total_pages': 3}, page_content='Saathi: Government Scheme Information Handbook  \n1. Pradhan Mantri Awas Yojana (PMAY) \nAbout the Scheme: \nAn initiative by the Government of India aimed at providing affordable housing for all by the'),
 Document(id='cbe0a7dc-45fc-4e35-b4f2-22aea305c445', metadata={'total_pages': 3, 'page': 0, 'creator': 'Microsoft® Word 2016', 'source': '/Users/adityaaryan/Documents/mlops/data/Saathi_Schemes_Info.pdf', 'producer': 'www.ilovepdf.com', 'author': 'python-docx', 'moddate': '2025-06-22T12:19:11+00:00', 'creationdate': '2025-06-22T12:19:11+00:00', 'page_label': '1'}, page_content='Saathi: Government Scheme Informat

In [91]:
# Method 2 - Using similarity search
question = "What are the eligibility criteria for PMAY scheme?"
retrieved_docs = vector_store.similarity_search(question, k=2) 
retrieved_docs

[Document(id='c627c775-a2a8-4415-8a1a-ebafb8906296', metadata={'author': 'python-docx', 'page_label': '2', 'producer': 'www.ilovepdf.com', 'creationdate': '2025-06-22T12:19:11+00:00', 'source': '/Users/adityaaryan/Documents/mlops/data/Saathi_Schemes_Info.pdf', 'total_pages': 3, 'moddate': '2025-06-22T12:19:11+00:00', 'creator': 'Microsoft® Word 2016', 'page': 1}, page_content='Indian citizens below the poverty line (BPL) \nAged 60+ (for old age pension), widowed women aged 40-59, or persons with 80%+ \ndisability \nNo regular source of income \nKey Benefits:'),
 Document(id='5719940c-20ee-414e-9ae7-6a713992b218', metadata={'page': 1, 'creator': 'Microsoft® Word 2016', 'moddate': '2025-06-22T12:19:11+00:00', 'page_label': '2', 'producer': 'www.ilovepdf.com', 'creationdate': '2025-06-22T12:19:11+00:00', 'author': 'python-docx', 'source': '/Users/adityaaryan/Documents/mlops/data/Saathi_Schemes_Info.pdf', 'total_pages': 3}, page_content='Indian citizens below the poverty line (BPL) \nAged 

In [81]:
#Convert retrieved docs into context text
context = "\n\n".join(doc.page_content for doc in docs)
context

'Saathi: Government Scheme Information Handbook  \n1. Pradhan Mantri Awas Yojana (PMAY) \nAbout the Scheme: \nAn initiative by the Government of India aimed at providing affordable housing for all by the\n\nSaathi: Government Scheme Information Handbook  \n1. Pradhan Mantri Awas Yojana (PMAY) \nAbout the Scheme: \nAn initiative by the Government of India aimed at providing affordable housing for all by the\n\nSaathi: Government Scheme Information Handbook  \n1. Pradhan Mantri Awas Yojana (PMAY) \nAbout the Scheme: \nAn initiative by the Government of India aimed at providing affordable housing for all by the'

In [82]:
prompt = f"""
You are a helpful assistant.
Answer the question using ONLY the context below.
If the answer is not present, say "I don't know".

Context:
{context}

Question:
{question}
"""

print(prompt)


You are a helpful assistant.
Answer the question using ONLY the context below.
If the answer is not present, say "I don't know".

Context:
Saathi: Government Scheme Information Handbook  
1. Pradhan Mantri Awas Yojana (PMAY) 
About the Scheme: 
An initiative by the Government of India aimed at providing affordable housing for all by the

Saathi: Government Scheme Information Handbook  
1. Pradhan Mantri Awas Yojana (PMAY) 
About the Scheme: 
An initiative by the Government of India aimed at providing affordable housing for all by the

Saathi: Government Scheme Information Handbook  
1. Pradhan Mantri Awas Yojana (PMAY) 
About the Scheme: 
An initiative by the Government of India aimed at providing affordable housing for all by the

Question:
What is PMAY scheme?



In [92]:
response=model.invoke(prompt)
print(response.content)

An initiative by the Government of India aimed at providing affordable housing for all.
